# Inverse graphics using graphics-based generative models and MCMC

### Installation:
Don't run this code (I'm not even sure if you can; there may be permission issues), 
but I am including it to show how we installed Mitsuba3 in our environment
which is by using a Conda environment

```julia
using Pkg
Pkg.activate("myenv")
Pkg.add("Conda")
using Conda
Conda.pip("install", "mitsuba")

```

In [2]:
using Pkg
Pkg.activate("myenv")
using Distributions
using ProgressMeter
using Gen, Plots
using Parameters
using PyCall
numpy = pyimport("numpy")
mi = pyimport("mitsuba")
mi.set_variant("scalar_rgb")

  Activating project at `~/Algorithms-of-the-Mind/labs/lab-05/myenv`


# Brief Mitsuba3 Introduction

Mitsuba3 is a physically-based graphics rendered. This means that instead of taking some of the common shortcuts (rasterization), it more faithfully captures the physical process by which light bounces of surfaces and creates images. This process is called ray-tracing, in which rays are simulated from the light sources in the scene, which bouncess off of the surfaces and gets sensed by a sensor (e.g., eye, camera) placed somewhere in the scene.

For our purposes Mitsuba3 serves as 

In [1]:
#use the teapot and transform it in various ways

# load a scene with a teapot
#scene = mi.load_file("./scenes/cbox.xml")

# read a data structure of what is in this scene 
#params = @pycall mi.traverse(scene)::PyObject

# Review this output and the xml file itself, which 

LoadError: UndefVarError: `mi` not defined

# Inverse graphics

We start by writing our generative model -- which in this case wraps a graphics engine 

In [3]:
@with_kw struct ModelParams
    path::String = "./cbox_generic.xml"
    scene::PyObject = mi.load_file(path)
    params::PyObject = @pycall mi.traverse(scene)::PyObject
    spp::Int32 = 16
end

function mitsuba_transform(modelparams, object, scale, translation)
    t = mi.Transform4f.translate(translation).scale([scale, scale, scale])
    set!(modelparams.params, object, t)
    println(get(modelparams.params, object))
end

function mitsuba_render(modelparams)
    image = mi.render(modelparams.scene, spp=modelparams.spp)
    bitmap = mi.Bitmap(image).convert(srgb_gamma=true)
end

mitsuba_render (generic function with 1 method)

In [4]:
@gen function room(modelparams::ModelParams)

    # prior over the scale of the left sphere
    s_left ~ uniform(0.1, 1.)
    # we assume we know a priori where this sphere will appear in the scene
    t_left = [-0.3, -0.5, 0.2]
    mitsuba_transform(modelparams, "left-object.to_world", s_left, t_left)
    
    # prior over the scale of the right sphere
    s_right ~ uniform(0.1, 1.)
    # we assume we know a priori where this sphere will appear in the scene
    t_right = [0.5, -0.75, -0.2] 
    mitsuba_transform(modelparams, "right-object.to_world", s_right, t_right)

    py"$(modelparams.params).update()"o

    # render the scene and get the output in an array
    bitmap = mitsuba_render(modelparams)
    mu = @pycall numpy.array(bitmap)::Array{Float32, 3}

    {:pred} ~ broadcasted_normal(mu, 1)

    return bitmap
    
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[ModelParams], false, Union{Nothing, Some{Any}}[nothing], var"##room#292", Bool[0], false)

In [5]:
modelparams = ModelParams()

ModelParams
  path: String "./cbox_generic.xml"
  scene: PyObject
  params: PyObject
  spp: Int32 16


In [6]:
trace = Gen.simulate(room, (modelparams,))
bitmap = Gen.get_retval(trace)

PyObject [[0.49555, 0, 0, -0.3],
 [0, 0.49555, 0, -0.5],
 [0, 0, 0.49555, 0.2],
 [0, 0, 0, 1]]
PyObject [[0.719129, 0, 0, 0.5],
 [0, 0.719129, 0, -0.75],
 [0, 0, 0.719129, -0.2],
 [0, 0, 0, 1]]


PyObject Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [128, 128],
  srgb_gamma = 1,
  struct = Struct<12>[
    float32 R; // @0, gamma, premultiplied alpha
    float32 G; // @4, gamma, premultiplied alpha
    float32 B; // @8, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

In [7]:
get_choices(trace)

│
├── :s_left : 0.49555040229332237
│
├── :pred : [-0.9341886411739548 1.2424849845840011 -1.4795806773603564 0.640602487893996 0.5740022503634542 -1.431496292335665 -0.490798668124835 1.1381399720710343 -0.5987451711968362 -0.6975144178629876 1.0479419104901837 0.9637107530563033 -0.6661703419829693 -0.8257974203677255 -0.6124121008500532 0.47197306466407535 1.8273694518679753 0.33434780236403866 0.30840094402431373 -1.5844808493263736 -0.3621209864639064 -1.025410415641444 -0.692522006805165 -0.3697408321324963 0.6267711610655989 0.2657217666257894 -2.3138083097801934 1.2835415449482728 1.5226220384047862 -0.16987896144417847 -0.680042643750135 0.1547164473217915 -0.01712391046255893 -0.07189295877629301 1.3429974683129333 -0.894950486903842 -0.6242476802604765 0.49260382684641985 0.48749567465158017 0.6055986624973178 0.12478829058456246 -1.0124099485836362 1.1891602658384233 0.2724312070849514 0.3684289181276815 -0.1841323209519603 0.48691533644350105 1.1704092001030053 0.710614140

### Now let's make an observed image using an entry in the Mitsuba gallery

In [8]:
obs_scene = ModelParams(path = "./scenes/cbox.xml", spp=64)
obs_bitmap = mitsuba_render(obs_scene)
obs_image = Gen.choicemap()
obs_image[:pred] = @pycall numpy.array(obs_bitmap)::Array{Float32, 3}
obs_scene = 1
# to view the image on your notebook
obs_bitmap

PyObject Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [128, 128],
  srgb_gamma = 1,
  struct = Struct<12>[
    float32 R; // @0, gamma, premultiplied alpha
    float32 G; // @4, gamma, premultiplied alpha
    float32 B; // @8, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

### Inference: Random walk MH

In [9]:
# include the truncated norm distribution
include("truncatednorm.jl")

In [10]:
# proposal distribution for the scale variables
@gen function scale_proposal(current_trace)
    # trunc_norm(mean, std, lower_bound, upper_bound)
    # why do we need a truncated norm, instead of a regular normal distribution?
    s_left ~ trunc_norm(current_trace[:s_left], 0.2, 0.1, 1.)
    s_right ~ trunc_norm(current_trace[:s_right], 0.2, 0.1, 1.)
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##scale_proposal#293", Bool[0], false)

In [11]:
function random_walk_mh(tr)
    
    # make a random-walk update on scale variables
    (tr, _) = mh(tr, scale_proposal, ())

    # return the updated trace
    tr
end

random_walk_mh (generic function with 1 method)

In [ ]:
t, = generate(room, (modelparams,), obs_image)

scores = Vector{Float64}(undef, 30)
for i in 1:30
    global t
    t = random_walk_mh(t)
    scores[i] = get_score(t)
    println(i)
end;

plot(scores)

PyObject [[0.854729, 0, 0, -0.3],
 [0, 0.854729, 0, -0.5],
 [0, 0, 0.854729, 0.2],
 [0, 0, 0, 1]]
PyObject [[0.565023, 0, 0, 0.5],
 [0, 0.565023, 0, -0.75],
 [0, 0, 0.565023, -0.2],
 [0, 0, 0, 1]]
PyObject [[0.83274, 0, 0, -0.3],
 [0, 0.83274, 0, -0.5],
 [0, 0, 0.83274, 0.2],
 [0, 0, 0, 1]]
PyObject [[0.601779, 0, 0, 0.5],
 [0, 0.601779, 0, -0.75],
 [0, 0, 0.601779, -0.2],
 [0, 0, 0, 1]]
1
PyObject [[0.551823, 0, 0, -0.3],
 [0, 0.551823, 0, -0.5],
 [0, 0, 0.551823, 0.2],
 [0, 0, 0, 1]]
PyObject [[0.869498, 0, 0, 0.5],
 [0, 0.869498, 0, -0.75],
 [0, 0, 0.869498, -0.2],
 [0, 0, 0, 1]]
2
PyObject [[0.340169, 0, 0, -0.3],
 [0, 0.340169, 0, -0.5],
 [0, 0, 0.340169, 0.2],
 [0, 0, 0, 1]]
PyObject [[0.69801, 0, 0, 0.5],
 [0, 0.69801, 0, -0.75],
 [0, 0, 0.69801, -0.2],
 [0, 0, 0, 1]]
3
PyObject [[0.333769, 0, 0, -0.3],
 [0, 0.333769, 0, -0.5],
 [0, 0, 0.333769, 0.2],
 [0, 0, 0, 1]]
PyObject [[0.632794, 0, 0, 0.5],
 [0, 0.632794, 0, -0.75],
 [0, 0, 0.632794, -0.2],
 [0, 0, 0, 1]]
4
PyObject [[0.

In [30]:
output_choicemap = Gen.choicemap()
output_choicemap[:s_left] = t[:s_left]
output_choicemap[:s_right] = t[:s_right]
output, = generate(room, (modelparams, ), output_choicemap)
bitmap = Gen.get_retval(output)

PyObject Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [128, 128],
  srgb_gamma = 1,
  struct = Struct<12>[
    float32 R; // @0, gamma, premultiplied alpha
    float32 G; // @4, gamma, premultiplied alpha
    float32 B; // @8, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]